In [1]:
#start with: folder with mag field
#- 10 folder with phi values - 10 folder x0 - 100 w/ theta value

In [5]:
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
import matplotlib.animation as animation
import os

testdir = "/home/darren/Desktop/plots/"
datadir = "/home/shared_data/"
plotdir = datadir+"plots/randomphi/"
mapdir = datadir+"Bmaps/"
date = "/6-20/"


newdir = datadir+'testdir'

In [13]:
#input N, return N equally spaced values between 0 and 2pi
def get_uniform_phi(N):
    phis = np.linspace(0, 2*math.pi, N)
    return phis
def get_uniform_theta(N):
    thetas = np.linspace(0, math.pi, N)
    return thetas

start_point = 3
end_point = 14
initial_B = 50 #(rougly 1% distortion at z = 3.0, 0% at z = 14)
final_B = 0

df_Mu2e = pd.read_pickle(mapdir+"Mu2e_DSMap_V13.p")
B_Mu2e_func = get_df_interp_func(mapdir+"Mu2e_DSMap_V13.p", gauss=False)

df_Mu2e_dis = get_B_df_distorted(df_Mu2e, v="0", Bz0 = initial_B, Bzf = 0, z0 = start_point, zf = end_point)
B_Mu2e_dis = get_df_interp_func(df=df_Mu2e_dis, gauss=False)

def run_solver(phi, theta, N_calc, field, xnaught, ynaught, znaught, name):
    ic_Mu2e = InitConds(t0=0., tf=4e-8, N_t=N_calc,
                    x0=xnaught, y0=ynaught, z0=znaught,
                    p0=104.96, theta0=theta, phi0=phi)
    e_solver = trajectory_solver(ic_Mu2e, B_func=field, bounds=bounds_Mu2e)
    sol = e_solver.solve_trajectory(verbose = False, atol=1e-10, rtol=1e-10) # high tolerance so it runs quickly for testing
    e_solver.dataframe['r'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    e_solver.to_pickle(datadir+f'50_Gauss_z_3-14/{phi}_{name}.pkl')
    
    return e_solver.dataframe

def savefiles(nphis, nthetas, x0s, N_calc): #x0s is an array of x0s u want
    phis = get_uniform_phi(nphis)
    thetas = get_uniform_theta(nthetas)
    nx0s = len(x0s)
    
    for i in phis:
        os.makedirs(newdir+f'/{i}')
        for j in thetas:
            os.makedirs(newdir+f'/{j}')
            for k in x0s:
                run_solver(i, j, n_calc, k, ynaught, znaught, 'nom')
                run_solver(i, j, n_calc, k, ynaught, znaught, 'dis')
                
                



In [14]:
savefiles(2, 2, [0.0, 0.5], 2000)

FileExistsError: [Errno 17] File exists: '/home/shared_data/testdir/0.0'

In [ ]:
#folder with bfield, x folders for phi values, 10 files in each folder with theta values
def readpickle(x, y):
    for phi in range(0, x, 1):
        for theta in range(0, y, 1):
            
        
    files = sorted(os.listdir(newdir))
    files[0], files[1], files[2], files[3] = files[2], files[3], files[0], files[1]

    phinom = []
    phidis = []
    nomdata = []
    disdata = []

    nom = {}
    dis = {}

    for file in files:
        if file.endswith('nom.pkl'):
            e_solvernom = trajectory_solver.from_pickle(newdir+'/'+file)
            phinom.append(e_solvernom.init_conds.phi0)
            nomdata.append(e_solvernom.dataframe)
            #nom.update({str(e_solvernom.init_conds.phi0), e_solvernom.dataframe})
            
        if file.endswith('dis.pkl'):
            e_solverdis = trajectory_solver.from_pickle(newdir+'/'+file)
            phidis.append(e_solverdis.init_conds.phi0)
            disdata.append(e_solverdis.dataframe)
            #dis.update({str(e_solverdis.init_conds.phi0), e_solverdis.dataframe})
    if phinom == phidis:
        for i in range(0, len(phinom), 1):
            nom.update({phinom[i]:nomdata[i]})
            dis.update({phidis[i]:disdata[i]})
            
    if phinom == phidis:
        return nom, dis, phinom, len(phinom)